# 🔀 Early Fusion Trainer - HuBERT Large + WavLM Base

**Architettura**: Concatena feature di HuBERT Large (1024D) e WavLM Base (768D) → CTC Head

**Requisiti**: GPU con ≥8GB VRAM (T4, RTX 3080+) con 4-bit quantization

**Modelli**:
- HuBERT Large: fine-tuned encoder
- WavLM Base: fine-tuned encoder

In [ ]:
import os, sys, zipfile, glob, re, shutil

def detect_environment():
    if 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules:
        return 'colab'
    elif '/kaggle' in os.getcwd() or 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        return 'kaggle'
    return 'local'

ENV = detect_environment()
print(f'🖥️ Ambiente: {ENV.upper()}')

In [ ]:
# COLAB Setup
if ENV == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')
    
    DRIVE_BACKUP = '/content/drive/MyDrive/phoneme_checkpoints'
    PROJECT_DIR = '/content/DeepLearning-Phoneme'
    ZIP_PATH = '/content/drive/MyDrive/DeepLearning-Phoneme.zip'
    
    # Model paths for Colab (adjust as needed)
    HUBERT_INPUT = '/content/drive/MyDrive/backup/hubert_large/final_model_hubert'
    WAVLM_INPUT = '/content/drive/MyDrive/backup/wavlm_base/final_model_aug_comb'
    
    if os.path.exists(ZIP_PATH):
        with zipfile.ZipFile(ZIP_PATH, 'r') as z:
            z.extractall('/content')
        print('✓ Extracted')
    else:
        raise FileNotFoundError(ZIP_PATH)
    
    os.makedirs(DRIVE_BACKUP, exist_ok=True)
    os.chdir(PROJECT_DIR)
    sys.path.insert(0, PROJECT_DIR)

In [ ]:
# KAGGLE Setup
if ENV == 'kaggle':
    import subprocess
    PROJECT_DIR = '/kaggle/working/pronuncIAtion'
    DRIVE_BACKUP = '/kaggle/working/checkpoints'
    
    # ✅ Path ai modelli fine-tuned caricati come dataset input
    MODEL_INPUT_DIR = '/kaggle/input/hubert-wavlmbase'  # Nome dataset Kaggle
    HUBERT_INPUT = f'{MODEL_INPUT_DIR}/hubert_large/final_model_hubert'
    WAVLM_INPUT = f'{MODEL_INPUT_DIR}/wavlm_base/final_model_aug_comb'
    
    # Clone/pull repo
    if not os.path.exists(PROJECT_DIR):
        subprocess.run(['git', 'clone', 'https://github.com/maurocarlu/pronuncIAtion.git', PROJECT_DIR])
    else:
        subprocess.run(['git', '-C', PROJECT_DIR, 'pull', 'origin', 'main'])
    
    # Data symlink
    DATA_INPUT = '/kaggle/input/pronunciation-data/data'
    DATA_TARGET = f'{PROJECT_DIR}/data'
    
    if os.path.islink(DATA_TARGET):
        print('✓ Data symlink exists')
    elif os.path.exists(DATA_TARGET):
        shutil.rmtree(DATA_TARGET)
        os.symlink(DATA_INPUT, DATA_TARGET)
    elif os.path.exists(DATA_INPUT):
        os.symlink(DATA_INPUT, DATA_TARGET)
    
    # Verify model paths
    print(f'\n📦 Modelli Input:')
    print(f'  HuBERT: {HUBERT_INPUT}')
    print(f'    Exists: {os.path.exists(HUBERT_INPUT)}')
    print(f'  WavLM:  {WAVLM_INPUT}')
    print(f'    Exists: {os.path.exists(WAVLM_INPUT)}')
    
    os.makedirs(DRIVE_BACKUP, exist_ok=True)
    os.chdir(PROJECT_DIR)
    sys.path.insert(0, PROJECT_DIR)
    print(f'\n✓ Kaggle ready: {PROJECT_DIR}')

In [ ]:
# LOCAL Setup
if ENV == 'local':
    PROJECT_DIR = os.getcwd()
    if 'notebooks' in PROJECT_DIR:
        PROJECT_DIR = os.path.dirname(PROJECT_DIR)
    DRIVE_BACKUP = f'{PROJECT_DIR}/outputs'
    
    # Model paths for local (adjust as needed)
    HUBERT_INPUT = f'{PROJECT_DIR}/outputs/hubert/final_model_hubert'
    WAVLM_INPUT = f'{PROJECT_DIR}/outputs/wavLM/final_model_aug_comb'

os.makedirs(DRIVE_BACKUP, exist_ok=True)
os.chdir(PROJECT_DIR)
sys.path.insert(0, PROJECT_DIR)
print(f'📁 Project: {PROJECT_DIR}')
print(f'💾 Checkpoints: {DRIVE_BACKUP}')

In [ ]:
# Install dependencies (including bitsandbytes for 4-bit quantization)
import subprocess
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 
                'transformers', 'datasets', 'evaluate', 'jiwer', 
                'soundfile', 'librosa', 'safetensors', 
                'bitsandbytes', 'accelerate'])  # Added for 4-bit

import torch
print(f'🔥 PyTorch {torch.__version__}, CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'📊 GPU: {torch.cuda.get_device_name(0)}')
    print(f'💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## ⚙️ Configuration

I path ai modelli fine-tuned sono configurati automaticamente in base all'ambiente (Kaggle/Colab/Local).

In [ ]:
# Usa i modelli fine-tuned dal path configurato sopra
CONFIG = {
    'csv_path': f'{PROJECT_DIR}/data/processed/combined_augmented.csv',
    'vocab_path': f'{PROJECT_DIR}/data/processed/vocab.json',
    'audio_base': PROJECT_DIR,
    'output_dir': f'{DRIVE_BACKUP}/early_fusion',
    
    # ✅ Modelli fine-tuned
    'hubert_path': HUBERT_INPUT,
    'wavlm_path': WAVLM_INPUT,
    
    # Training params - optimized for T4 (16GB) with 4-bit quantization
    'epochs': 5,
    'batch_size': 1,  # Small for memory
    'gradient_accumulation': 16,  # Effective batch = 16
    'learning_rate': 1e-4,
}

print('📋 Configuration:')
for k,v in CONFIG.items():
    if 'path' in k or 'dir' in k:
        status = '✓' if os.path.exists(str(v)) else '✗'
        print(f'  {status} {k}: {v}')
    else:
        print(f'  • {k}: {v}')

## 🚀 Training Early Fusion

**Architecture**:
```
Audio → HuBERT Large (frozen, fine-tuned) → 1024D ─┐
                                                   ├→ concat → CTC Head → IPA
Audio → WavLM Base (frozen, fine-tuned)  → 768D ──┘
```

**Memory con 4-bit quantization**:
- HuBERT Large: ~1.5GB
- WavLM Base: ~0.5GB
- **Totale: ~6-8GB** (fattibile su T4!)

In [ ]:
# 🔀 EARLY FUSION TRAINING
import subprocess

cmd = [
    sys.executable, 'scripts/training/train_early_fusion.py',
    '--data-csv', CONFIG['csv_path'],
    '--vocab-path', CONFIG['vocab_path'],
    '--audio-base', CONFIG['audio_base'],
    '--output-dir', CONFIG['output_dir'],
    '--epochs', str(CONFIG['epochs']),
    '--batch-size', str(CONFIG['batch_size']),
    '--gradient-accumulation', str(CONFIG['gradient_accumulation']),
    '--learning-rate', str(CONFIG['learning_rate']),
    '--hubert-path', CONFIG['hubert_path'],
    '--wavlm-path', CONFIG['wavlm_path'],
]

print('🚀 Running command:')
print(' '.join(cmd))
print()

result = subprocess.run(cmd, capture_output=False)

if result.returncode == 0:
    print('\n✅ Training completato!')
else:
    print(f'\n❌ Training fallito con codice {result.returncode}')

## 🔄 Resume Training (optional)

Se il training è stato interrotto, esegui questa cella per riprendere dall'ultimo checkpoint.

In [ ]:
# 🔄 RESUME FROM CHECKPOINT
import subprocess

cmd = [
    sys.executable, 'scripts/training/train_early_fusion.py',
    '--data-csv', CONFIG['csv_path'],
    '--vocab-path', CONFIG['vocab_path'],
    '--audio-base', CONFIG['audio_base'],
    '--output-dir', CONFIG['output_dir'],
    '--epochs', str(CONFIG['epochs']),
    '--batch-size', str(CONFIG['batch_size']),
    '--hubert-path', CONFIG['hubert_path'],
    '--wavlm-path', CONFIG['wavlm_path'],
    '--resume',  # Resume flag
]

result = subprocess.run(cmd, capture_output=False)
print('✅ Resume completato!' if result.returncode == 0 else f'❌ Errore {result.returncode}')

## 📊 Check Training Output

In [ ]:
# List checkpoints
output_dir = CONFIG['output_dir']
if os.path.exists(output_dir):
    files = sorted(os.listdir(output_dir))
    print(f'📁 {output_dir}:')
    for f in files:
        path = os.path.join(output_dir, f)
        if os.path.isdir(path):
            size = sum(os.path.getsize(os.path.join(path, x)) for x in os.listdir(path) if os.path.isfile(os.path.join(path, x)))
            print(f'  📂 {f} ({size/1e6:.1f} MB)')
        else:
            print(f'  📄 {f}')
else:
    print(f'❌ Output dir non esiste: {output_dir}')

## 💾 Download/Backup

In [ ]:
# Create ZIP of final model
import datetime

final_model = f"{CONFIG['output_dir']}/final_model_early_fusion"

if os.path.exists(final_model):
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
    zip_name = f'early_fusion_final_{timestamp}'
    
    if ENV == 'kaggle':
        zip_path = f'/kaggle/working/{zip_name}'
    else:
        zip_path = f'/content/{zip_name}' if ENV == 'colab' else f'{DRIVE_BACKUP}/{zip_name}'
    
    print(f'📦 Creating ZIP: {zip_name}.zip')
    shutil.make_archive(zip_path, 'zip', final_model)
    print(f'✓ Created: {zip_path}.zip')
    
    if ENV == 'colab':
        from google.colab import files
        files.download(f'{zip_path}.zip')
else:
    print(f'❌ Final model not found: {final_model}')
    print('💡 Run training first or check for checkpoints')

In [ ]:
# Cleanup disk (Kaggle)
if ENV == 'kaggle':
    for f in ['/root/.cache/huggingface']:
        if os.path.exists(f) and not os.path.islink(f):
            shutil.rmtree(f)
            print(f'🗑️ {f}')